In [ ]:
# Auto reload modules
%load_ext autoreload
%autoreload 2
#if you haven't installed vsf and just want to run this from the current directory, uncomment the following lines
import sys
sys.path.append('..')

### Download the demo data from box (130MB): 
+ #### Demo data: https://uofi.box.com/s/31wozq63qgqvg8012r1jdfj5mdchmmfp

### Demo data include 6 objects with heterogeneous stiffness:  

- **brown_boot_moving**:  
    - **Description**: a boot placed freely on the table. Punyo sensor + kinova robot arm is used to collect tactile data. Use FoundationPose to track 6D pose of the object.  
    - **Support sensor(s)**: punyo dense force (suggested), punyo pressure, joint torque.  
    - **VSF initialized from**: object mesh  
- **brown_boot_fixed**:  
    - **Description**: a boot mounted on the table. Punyo sensor + kinova robot arm is used to collect tactile data.  
    - **Support sensor(s)**: punyo dense force (suggested), punyo pressure, joint torque.  
    - **VSF initialized from**: object RGBD image (suggested), object bounding box  
- **gray_shoe_fixed**:  
    - same as above  
- **white_nike_fixed**:  
    - same as above  
- **rubber_fig_tall_angle00**:  
    - **Description**: an artificial tree placed on the ground. Use kinova arm to push the tree to collect the tactile data.  
    - **Support sensor(s)**: joint torque.  
    - **VSF initialized from**: object RGBD image  
- **rubber_fig_tall_angle06**:  
    - same as above  


### Visualize VSF

We have already estimated some neural VSFs for you in the demo dataset.  To visualize them, you can use the `vsf_show` function.

In [ ]:
import os

# demo data directory
DEMO_DIR = "../demo_data" # change to your path

OBJECT = "brown_boot_fixed"
DATA_DIR = os.path.join(DEMO_DIR, 'datasets', OBJECT)

In [ ]:
# visualize VSF
from vsf.constructors import vsf_from_file
import torch

#estimated VSFs have been saved to the "outputs" folder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vsf = vsf_from_file(os.path.join(DEMO_DIR, "saved_vsfs", "brown_boot_moving", "neural_vsf.pt")).to(device)

from klampt import vis
from vsf.visualize.klampt_visualization import vsf_show
vis.init('PyQt5')  #needed inside Jupyter Notebook to show an OpenGL window
vsf_show(vsf)

### Walkthrough neural VSF training and visualization

### Create a world and simulator

In [ ]:
import numpy as np
import json
import klampt
from klampt.math import se3
from vsf.sim import QuasistaticVSFSimulator
from vsf.sensor.punyo_dense_force_sensor import PunyoDenseForceSensor
from vsf.sim.klampt_world_wrapper import klamptWorldWrapper
from vsf.utils.klampt_utils import load_trimesh_preserve_vertices

# NOTE: This part of the code shows how to manually setup a simulation world with a robot, a tactile sensor and a deformable object,
#       and how to estimate the VSF from tactile data.
#       A script to automate this process from config files is in scripts/neural_vsf_estimate.py

# create a world for simulation and visualization
world = klamptWorldWrapper()

# add robot/mesh to the world
world.add_robot('kinova', os.path.join('../knowledge/robot_model', 'kinova_gen3.urdf'))
robot = world.world.robot(0)
ee_link = robot.link(robot.numLinks()-1)
punyo2ee = np.array(json.load(open(os.path.join('../knowledge/robot_model', 'punyo2end_effector_transform.json')))['punyo2ee'])

#this function should be used instead of native Klampt loaders due to a known Assimp configuration issue
mesh = load_trimesh_preserve_vertices(os.path.join('../knowledge/robot_model', 'punyo_mesh_partial.ply'))
world.add_geometry('punyo', mesh, 'deformable', ee_link.getName(), punyo2ee)

# adds the VSF object to the world, for visualization purposes
# add object to the visualization without interfering with the simulator
object = klampt.Geometry3D()

# visualize the object with a heightmap (projected from RGB-D images)
import imageio
rgb_image = imageio.imread(os.path.join(DATA_DIR, "object", "color_img.jpg"))
depth_image = imageio.imread(os.path.join(DATA_DIR, "object", "depth_img.png"))
depth_scale = 1000.0
depth_trunc = 2.0

intrinsic = json.load(open(os.path.join(DATA_DIR, "object", "intrinsic.json")))
extrinsic = json.load(open(os.path.join(DATA_DIR, "object", "extrinsic.json")))
extrinsic = np.array(extrinsic['cam2world'])

bmin, bmax = np.load(os.path.join(DATA_DIR, "object", "aabb.npy"))
vp = klampt.Viewport()
vp.setPose(*se3.from_homogeneous(extrinsic))
vp.w, vp.h = rgb_image.shape[1], rgb_image.shape[0]
vp.fx, vp.fy = intrinsic['fx'], intrinsic['fy']
vp.cx, vp.cy = intrinsic['cx'], intrinsic['cy']

# remove points beyond the truncation distance    
depth_image[depth_image > depth_trunc*depth_scale] = 0

# create a heightmap from the rgbd image
hm_data = klampt.Heightmap()
hm_data.setViewport(vp)
hm_data.setHeightImage(depth_image, 1/depth_scale)
hm_data.setColorImage(rgb_image)
object.setHeightmap(hm_data)
object = object.convert('PointCloud')

# initialize sensors
sensors = [PunyoDenseForceSensor('punyo_force', 'punyo')] # add sensor named 'punyo_force' and attach it to the mesh named 'punyo'

# create simulator
sim = QuasistaticVSFSimulator(world, sensors)

### Load tactile dataset and visualize it

In [ ]:
import dacite
from vsf.dataset.constructors import dataset_from_config, DatasetConfig

from vsf.utils.config_utils import load_config_recursive
dataset_config = load_config_recursive(os.path.join('../demo_data/datasets/punyo_dataset_config.yaml'))
dataset_config = dacite.from_dict(DatasetConfig, dataset_config)
dataset_config.path = os.path.join('../demo_data/datasets', OBJECT)
dataset = dataset_from_config(dataset_config)

In [ ]:
from klampt import vis
vis.add("world", world.world)

# add table
# the table are for visualization only, not added to the simulator to avoid extra computation in the simulation
from klampt.model.create import box
b1 = box(1.,1.5,1.1,center=(0.25,0,-0.55),type='GeometricPrimitive')
vis.add("table", b1, hide_label=True)
b2 = box(0.7,0.45,0.12,center=(0.3,0.4,0),type='GeometricPrimitive')
vis.add("box", b2, hide_label=True)

# add visualization object to show its estimated pose
vis.add("object", object)
vis.show()

import time
for i in range(len(dataset)):
    seq = dataset[i]
    for frame in seq:
        control = {}
        control['kinova'] = frame['kinova_state']
        control['punyo'] = frame['punyo_state']

        # step simulation
        vis.lock()
        sim.step(control, 0.1)
        vis.unlock()

        time.sleep(2/len(seq)) # visualize each sequence for 2 second

    sim.reset()
    if i >= 5:
        break
vis.scene().clear()

In [ ]:
# create VSF model
import torch
from vsf.constructors import vsf_from_box

# create vsf from a bounding box
aabb = np.load(os.path.join(DATA_DIR, "object", "aabb.npy"))
vsf = vsf_from_box(aabb[0], aabb[1], type='neural')

# move vsf to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vsf = vsf.to(device)

# add vsf to the scene
sim.add_deformable('vsf',vsf)

## Run estimation

The following code runs a batch estimation over the whole dataset.  Note that training is performed by randomizing over sequences in the dataset.

In [ ]:
# some sensors need to be calibrated before use
# for PunyoDenseForceSensor, no calibration is needed
calibrators = {}

In [ ]:
# Batch VSF estimation demo
from vsf.estimator.neural_vsf_estimator import NeuralVSFEstimator, NeuralVSFEstimatorConfig

# create estimator
estimator_config = NeuralVSFEstimatorConfig(lr=2e-4,
                                            regularizer_samples=500,
                                            regularizer_scale=1e-8,
                                            max_epochs=500,
                                            down_sample_rate=1)
                                            
estimator = NeuralVSFEstimator(estimator_config)

print("Starting batch estimation, using device",vsf.device)
estimator.batch_estimate(sim, vsf, dataset, dataset_config, calibrators)

The following code runs an online estimation for each sequence in the dataset.  This does have a risk of catastrophic forgetting, since there is no memory replay buffer (as in Point VSF online estimators).

In [ ]:
# Online VSF estimation demo
from vsf.utils.data_utils import remap_dict_in_seq
# create estimator
estimator_config = NeuralVSFEstimatorConfig(lr=1e-3,
                                            regularizer_samples=500,
                                            regularizer_scale=1e-8,
                                            down_sample_rate=1)
estimator = NeuralVSFEstimator(estimator_config)

estimator.online_init(sim, vsf)

dt = 0.1
for i in range(len(dataset)):
    print("Beginning sequence",i)
    seq = dataset[i]
    sim.reset()
    estimator.online_reset(sim)

    control_keys = sim.get_control_keys()
    sensor_keys = sim.get_sensor_keys()
    control_seq, observation_seq = remap_dict_in_seq(seq, control_keys, sensor_keys)

    #calibrate the sensors from the sequences
    ncalibrate = 0
    if calibrators is not None and calibrators != {}:
        for k,v in calibrators.items():
            sensor = sim.get_sensor(k)
            if sensor is None: raise ValueError(f"Sensor {k} not found in simulator")
            n = v.calibrate(sensor, sim, control_seq, observation_seq)
            ncalibrate = max(n,ncalibrate)


    for i in range(ncalibrate, len(seq), estimator_config.down_sample_rate):
        controls = control_seq[i]
        observations = observation_seq[i]
        sim.step(controls,dt)
    
        estimator.online_update(sim, dt, observations)

estimator.online_finalize()


In [ ]:
# visualize VSF
from vsf.visualize.klampt_visualization import vsf_show
vsf_show(vsf)

In [ ]:
# save to disk        
os.makedirs(os.path.join(DEMO_DIR, "saved_vsfs", OBJECT), exist_ok=True)
vsf.save(os.path.join(DEMO_DIR, "saved_vsfs", OBJECT, "neural_vsf_playground.pt"))

### Different VSF initialization method and VSF pose control

VSF can be initialized from different object information. The above example uses a simple bounding box to initialize a Neural VSF. Other initialization methods provided including initializing from object mesh / object RGBD image. These two methods eliminate empty space, so can result in better training speed and model quality. 

Pose control for VSF is supported in simulator. This allows the target object to be moving while collecting the data.

Here is an example for reconstructing Nerual VSF for an boot lying freely on a table, with object mesh for VSF initialization.

In [ ]:
OBJECT = "brown_boot_moving"
DATA_DIR = os.path.join(DEMO_DIR, 'datasets', OBJECT)
print("Loading dataset from", DATA_DIR)

In [ ]:
world = klamptWorldWrapper()

# add robot/mesh to the world
world.add_robot('kinova', os.path.join('../knowledge/robot_model', 'kinova_gen3.urdf'))
robot = world.world.robot(0)
ee_link = robot.link(robot.numLinks()-1)
punyo2ee = np.array(json.load(open(os.path.join('../knowledge/robot_model', 'punyo2end_effector_transform.json')))['punyo2ee'])

#this function should be used instead of native Klampt loaders due to a known Assimp configuration issue
mesh = load_trimesh_preserve_vertices(os.path.join('../knowledge/robot_model', 'punyo_mesh_partial.ply'))
world.add_geometry('punyo', mesh, 'deformable', ee_link.getName(), punyo2ee)

# load the object mesh , for visualization purposes
object = klampt.Geometry3D()
object.loadFile(os.path.join(DATA_DIR, "object", "mesh.obj"))

# initialize sensors
sensors = [PunyoDenseForceSensor('punyo_force', 'punyo')] # add sensor named 'punyo_force' and attach it to the mesh named 'punyo'


# create simulator
# use the same world and sensors config as before
sim = QuasistaticVSFSimulator(world, sensors)

In [ ]:
dataset_config.path = os.path.join('../demo_data/datasets', OBJECT)
dataset = dataset_from_config(dataset_config)

In [ ]:
from klampt import vis
vis.add("world", world.world)


# add table
# the table are for visualization only, not added to the simulator to avoid extra computation in the simulation
from klampt.model.create import box
b1 = box(1.,1.5,1.1,center=(0.25,0,-0.55),type='GeometricPrimitive')
vis.add("table", b1, hide_label=True)
b2 = box(0.7,0.45,0.12,center=(0.3,0.4,0),type='GeometricPrimitive')
vis.add("box", b2, hide_label=True)

# add visualization object to show its estimated pose
vis.add("object", object)
vis.show()

import time
for i in range(len(dataset)):
    seq = dataset[i]
    for frame in seq:
        control = {}
        control['kinova'] = frame['kinova_state']
        control['punyo'] = frame['punyo_state']

        # step simulation
        vis.lock()
        sim.step(control, 0.1)
        object.setCurrentTransform(*se3.from_ndarray(frame['vsf_state'])) # object mesh is for visualization only, so not added to the simulator
        vis.unlock()

        time.sleep(2/len(seq)) # visualize each sequence for 2 second

    sim.reset()
    if i >= 5:
        break
vis.clear()

In [ ]:
from vsf.constructors import vsf_from_mesh

# create vsf from a mesh -- this will do a better job estimating stiffness at the object boundaries
vsf = vsf_from_mesh(os.path.join(DATA_DIR, "object", "mesh.obj"))

# move vsf to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vsf = vsf.to(device)

# add vsf to the scene
sim.add_deformable('vsf',vsf)

In [ ]:
# use the same estimator config as before
print("Starting batch estimation, using device",vsf.device)
estimator.batch_estimate(sim, vsf, dataset, dataset_config, calibrators)

In [ ]:
# visualize VSF
from vsf.visualize.klampt_visualization import vsf_show
vsf_show(vsf)

### Neural VSF to Point VSF

Neural VSF can be converted to Point VSF. The following code shows how to convert a neural VSF to a Point VSF. The Point VSF is initialized by sampling points from the neural VSF.

In [ ]:
from vsf.constructors import vsf_from_vsf, PointVSFConfig

voxel_size = 3e-3
point_config = PointVSFConfig(voxel_size=voxel_size)
point_vsf = vsf_from_vsf(point_config, vsf)
vsf_show(point_vsf)

### Point VSF to Neural VSF

Similarly, Point VSF can be converted to Neural VSF. The following code shows how to convert a Point VSF to a Neural VSF. The neural VSF is trained by distillation.

In [ ]:
from vsf.constructors import vsf_from_vsf, NeuralVSFConfig

neural_config = NeuralVSFConfig()

convert_config = NeuralVSFEstimatorConfig(lr=2e-4,
                                          regularizer_samples=1000,
                                          regularizer_scale=1e2,
                                          max_epochs=500)
neural_vsf = vsf_from_vsf(neural_config, point_vsf, convert_config)
vsf_show(neural_vsf)